In [1]:
import os
import urllib.parse
import urllib.request
import xml.etree.ElementTree as ET

In [2]:
def get_image_urls(ensembl_input_url):
    req = urllib.request.Request(ensembl_input_url)
    xml = urllib.request.urlopen(req).read()
    doc = ET.fromstring(xml.decode("utf-8"))
    urls = []
    for proteinAtlas in doc.iter('proteinAtlas'):
        for cellExpression in proteinAtlas.iter('cellExpression'):
            for imageUrl in cellExpression.iter('imageUrl'):
                if 'green' in imageUrl.text:
                    urls += [imageUrl.text]
    return urls

In [3]:
def get_single_channel_urls(multi_channel_url):
    parname, bname = os.path.split(multi_channel_url)
    name, ext = bname.split('.')
    plate, well, field = name.split("_")[:3]
    channels = name.split("_")[3:]
    single_channel_urls = []
    for channel in channels:
        sc_name = '_'.join([plate, well, field, channel])
        sc_bname = '.'.join([sc_name,ext])
        sc_url = os.path.join(parname,sc_bname)
        single_channel_urls += [sc_url]
    return single_channel_urls

In [4]:
urls = get_image_urls('https://www.proteinatlas.org/ENSG00000134057.xml')
[get_single_channel_urls(url) for url in urls]

[['http://v18.proteinatlas.org/images/115/672_E2_1_blue.jpg',
  'http://v18.proteinatlas.org/images/115/672_E2_1_red.jpg',
  'http://v18.proteinatlas.org/images/115/672_E2_1_green.jpg'],
 ['http://v18.proteinatlas.org/images/115/672_E2_2_blue.jpg',
  'http://v18.proteinatlas.org/images/115/672_E2_2_red.jpg',
  'http://v18.proteinatlas.org/images/115/672_E2_2_green.jpg'],
 ['http://v18.proteinatlas.org/images/115/663_E2_1_blue.jpg',
  'http://v18.proteinatlas.org/images/115/663_E2_1_red.jpg',
  'http://v18.proteinatlas.org/images/115/663_E2_1_green.jpg'],
 ['http://v18.proteinatlas.org/images/115/663_E2_5_blue.jpg',
  'http://v18.proteinatlas.org/images/115/663_E2_5_red.jpg',
  'http://v18.proteinatlas.org/images/115/663_E2_5_green.jpg'],
 ['http://v18.proteinatlas.org/images/115/646_E2_1_blue.jpg',
  'http://v18.proteinatlas.org/images/115/646_E2_1_red.jpg',
  'http://v18.proteinatlas.org/images/115/646_E2_1_green.jpg'],
 ['http://v18.proteinatlas.org/images/115/646_E2_2_blue.jpg',
  '